In [ ]:
!pip install requirements.txt

In [26]:
%load_ext autoreload
%autoreload 2

from flask import Flask
from flask.helpers import get_root_path

from jupyter_dash import JupyterDash
from dash_app.layout import layout
from dash_app.callbacks import register_callbacks

app = JupyterDash(__name__, #server=server,
                  url_base_pathname='/',
                  assets_folder='dash_app/assets/')

app.title = 'Gait Analyzer'
app.layout = layout
register_callbacks(app)

# Run app and display result inline in the notebook
app.run_server(mode='inline', debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from plotly.subplots import make_subplots
fig = make_subplots(2, 1, shared_xaxes=True)
for i, n in enumerate(['r_knee_flex', 'l_knee_flex']):
    fig.add_trace(go.Scatter(y=angles[n], name=n), i+1, 1)
fig.update_yaxes(matches='y')
fig.update_layout(dragmode= 'pan', xaxis=dict(range=[0,300]), yaxis=dict(range=[120,190], fixedrange=True))
fig.add_shape(
    dict(type="line", x0=20, x1=20, y0=120, y1=200, line_color="green"), row="all", col=1
)
fig.show()
#fig.update_shapes(dict(x0=30, x1=30))

NameError: name 'go' is not defined